In [ ]:
# !pip install -U transformers

In [2]:
import torch
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import warnings
import csv
import os
warnings.filterwarnings("ignore")

# 设备配置（自动适配GPU/CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# 模型路径（保持你原来的不变）
model_path = "/kaggle/input/llama3-8b-instruct/transformers/default/1/Meta-Llama-3-8B-Instruct"

# ===================== 只加载一次Tokenizer和Model（核心修改：删除重复加载）=====================
# 加载Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
# 设置pad_token和padding_side（Llama 3默认没有pad_token，用eos_token替代）
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16, 
    device_map="auto", 
    low_cpu_mem_usage=True,  
    trust_remote_code=True 
)

# 模型设置为评估模式
model.eval()

print("✅ Model and Tokenizer loaded successfully!")
print(f"📊 配置：float16精度 + 自动设备分配 + hidden_states启用")

2025-11-10 12:10:48.368474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762776648.555083      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762776648.611232      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model and Tokenizer loaded successfully!
📊 配置：float16精度 + 自动设备分配 + hidden_states启用


In [ ]:
import os

# 检查文件是否存在
csv_path = "/kaggle/input/testset/testset.csv"
if not os.path.exists(csv_path):
    print(f"文件不存在！请检查路径：{csv_path}")
    print("当前目录下的文件列表：", os.listdir("/kaggle/input/testset"))  # 打印文件夹内的文件
else:
    print("文件存在，可以正常加载")

### testset

In [ ]:
# 全局变量保存实验一数据
experiment1_data = {}


def save_experiment1_data(detailed_result):
    """保存实验一的hidden states数据"""
    test_id = detailed_result['test_id']
    
    # 构建保存的数据结构
    saved_data = {
        'test_id': test_id,
        'problem': detailed_result.get('problem', ''),
        'step_by_step_response': detailed_result['step_by_step_response'],
        'step_data': [],
        'skipping_info': detailed_result['skipping_info'],
        'all_skip_results': detailed_result['all_skip_results'],
        'hidden_states_info': {
            'total_steps': len(detailed_result.get('step_data', [])),
            'has_skipped_steps': len(detailed_result['skipping_info']['skipped_steps']) > 0,
            'saved_timestamp': str(np.datetime64('now')),
            'target_layer': -1  # 新增：记录提取隐藏状态的层（-1表示最后一层）
        }
    }
    
    # 保存步骤数据（包括hidden states和layer_idx）
    for step in detailed_result.get('step_data', []):
        step_data_copy = step.copy()
        # 确保hidden state是numpy数组
        if 'hidden_state' in step_data_copy:
            step_data_copy['hidden_state'] = step_data_copy['hidden_state']
        # 新增：保存层索引（与实验三Tuned Lens匹配）
        step_data_copy['layer_idx'] = step.get('layer_idx', -1)
        saved_data['step_data'].append(step_data_copy)
    
    # 保存到全局变量
    experiment1_data[test_id] = saved_data
    
    # 保存到文件
    filename = f"experiment1_hidden_states_{test_id}.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(saved_data, f)
    
    # 打印保存信息
    print(f"✓ 已保存 {test_id} 的hidden states数据到 {filename}")
    print(f"  - 步骤数量: {len(saved_data['step_data'])}")
    print(f"  - 跳步数量: {len(saved_data['skipping_info']['skipped_steps'])}")
    if saved_data['step_data']:
        print(f"  - hidden state形状: {saved_data['step_data'][0]['hidden_state'].shape}")
        print(f"  - 提取层索引: {saved_data['step_data'][0]['layer_idx']}")  # 新增：打印层索引

def save_complete_experiment1_data():
    """保存完整的实验一数据集"""
    if experiment1_data:
        with open("experiment1_complete_data.pkl", 'wb') as f:
            pickle.dump(experiment1_data, f)
        print(f"\n🎉 已保存完整的实验一数据，共 {len(experiment1_data)} 个测试案例")
        return True
    else:
        print("❌ 没有实验数据可保存")
        return False

def summarize_experiment1_data():
    """汇总实验一数据"""
    print("\n" + "="*60)
    print("实验一数据汇总")
    print("="*60)
    
    if not experiment1_data:
        print("暂无实验数据")
        return
    
    for test_id, data in experiment1_data.items():
        print(f"\n{test_id}:")
        print(f"  问题: {data['problem'][:100]}...")
        print(f"  推理步骤: {len(data['step_data'])}")
        print(f"  跳步: {len(data['skipping_info']['skipped_steps'])}")
        print(f"  跳步详情: {data['skipping_info']['skipped_steps']}")
        if data['step_data']:
            print(f"  提取层索引: {data['step_data'][0]['layer_idx']}")  # 新增：显示层索引

def calculate_similarity_between_paths(actual_skip_hidden, generated_paths_hidden_states):
    """
    计算实际跳步hidden states与生成路径hidden states的相似度
    
    Args:
        actual_skip_hidden: 实际跳步步骤的hidden state (numpy array)
        generated_paths_hidden_states: 生成的各路径hidden states列表
    
    Returns:
        similarities: 各路径的相似度列表
    """
    similarities = []
    actual_vector = actual_skip_hidden.reshape(1, -1)
    
    for path_hidden in generated_paths_hidden_states:
        if path_hidden is not None:
            path_vector = path_hidden.reshape(1, -1)
            sim = cosine_similarity(actual_vector, path_vector)[0][0]
            similarities.append(float(sim))
        else:
            similarities.append(0.0)
    
    return similarities

def get_actual_skip_hidden_state(skip_step_key, step_data, skipping_info, outputs, prompt_length):
    """
    获取实际跳步步骤的hidden state（新增layer_idx返回）
    
    Args:
        skip_step_key: 跳步的步骤key (如 'step_2')
        step_data: 步骤数据
        skipping_info: 跳步信息
        outputs: 模型输出
        prompt_length: prompt长度
    
    Returns:
        hidden_state: 跳步步骤的hidden state
        layer_idx: 对应的层索引
    """
    # 方法1: 尝试从step_data中直接找到跳步步骤
    for step in step_data:
        if step['step_number'] == skip_step_key:
            return step['hidden_state'], step.get('layer_idx', -1)  # 新增：返回layer_idx
    
    # 方法2: 如果找不到，根据跳步位置推断
    if step_data:
        return step_data[-1]['hidden_state'], step_data[-1].get('layer_idx', -1)  # 新增：返回layer_idx
    
    # 方法3: 返回整个推理的最后一个hidden state
    generated_ids = outputs.sequences[0, prompt_length:]
    layer_idx = -1  # 默认最后一层
    if len(generated_ids) > 0:
        actual_hidden_idx = prompt_length + len(generated_ids) - 1
        if actual_hidden_idx < len(outputs.hidden_states[layer_idx]):
            hidden_state = outputs.hidden_states[layer_idx][actual_hidden_idx][0, -1, :].cpu().numpy()
            return hidden_state, layer_idx
    
    return None, -1

def save_hidden_states(data_dict, filename="hidden_states_data.pkl"):
    """保存数据"""
    with open(filename, 'wb') as f:
        pickle.dump(data_dict, f)

def build_llama3_prompt(system_msg, user_msg):
    """构建符合Llama 3格式的prompt"""
    return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

# =====================================================
# 读取CSV测试集函数
# =====================================================
def load_testset(csv_path="/kaggle/input/testset/testset.csv"):
    """读取CSV格式的测试集"""
    testset = []
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            expected_steps = row['expected_complete_reasoning'].split('||')
            testset.append({
                'id': row['id'],
                'problem': row['problem'],
                'expected_complete_reasoning': expected_steps
            })
    print(f"✓ 成功加载 {len(testset)} 个测试问题")
    return testset

# =====================================================
# 步骤级hidden states提取
# =====================================================
def extract_step_level_hidden_states(outputs, prompt_length, generated_text):
    steps = parse_step_structure(generated_text)
    step_data = []
    generated_ids = outputs.sequences[0, prompt_length:]
    full_generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    total_hidden_length = len(generated_ids)

    layer_idx = -1
    
    current_position = 0
    for step_num, step_text in steps.items():
        step_text_clean = step_text.strip()
        step_start = full_generated_text.find(step_text_clean, current_position)
        
        if step_start == -1:
            full_text_clean = " ".join(full_generated_text.split())
            step_text_clean_no_space = " ".join(step_text_clean.split())
            step_start = full_text_clean.find(step_text_clean_no_space, current_position)
            if step_start == -1:
                continue
        
        step_tokens = tokenizer.encode(step_text_clean, add_special_tokens=False)
        step_token_length = len(step_tokens)
        if step_token_length == 0:
            continue
        
        text_before_step = full_generated_text[:step_start]
        tokens_before_step = tokenizer.encode(text_before_step, add_special_tokens=False)
        step_start_token_idx = len(tokens_before_step)
        step_end_token_idx = step_start_token_idx + step_token_length - 1
        
        if step_end_token_idx < step_start_token_idx:
            current_position = step_start + len(step_text_clean)
            continue
        
        if step_end_token_idx >= total_hidden_length:
            step_end_token_idx = total_hidden_length - 1
        
        actual_hidden_idx = prompt_length + step_end_token_idx
        if actual_hidden_idx >= len(outputs.hidden_states[-1]):
            actual_hidden_idx = len(outputs.hidden_states[-1]) - 1
        step_hidden = outputs.hidden_states[-1][actual_hidden_idx][0, -1, :].cpu().numpy()
        
        step_data.append({
            'step_number': step_num,
            'step_text': step_text,
            'hidden_state': step_hidden,
            'layer_idx': layer_idx, 
            'token_start': step_start_token_idx,
            'token_end': step_end_token_idx,
            'token_length': step_token_length,
            'text_position': step_start
        })
        
        current_position = step_start + len(step_text_clean)
    
    if not step_data and len(generated_ids) > 0:
        actual_hidden_idx = prompt_length + len(generated_ids) - 1
        if actual_hidden_idx >= len(outputs.hidden_states[-1]):
            actual_hidden_idx = len(outputs.hidden_states[-1]) - 1
        last_hidden = outputs.hidden_states[-1][actual_hidden_idx][0, -1, :].cpu().numpy()
        step_data.append({
            'step_number': 'step_1',
            'step_text': generated_text,
            'hidden_state': last_hidden,
            'layer_idx': layer_idx, 
            'token_start': 0,
            'token_end': len(generated_ids) - 1,
            'token_length': len(generated_ids),
            'text_position': 0
        })
    
    return step_data

def parse_step_structure(text):
    steps = {}
    step_patterns = [
        r'(Step\s*\d+\s*:)(.*?)(?=Step\s*\d+\s*:|Final Answer:|$)',
        r'(\d+\.\s*)(.*?)(?=\d+\.\s*|Final Answer:|$)',
        r'(\d+\)\s*)(.*?)(?=\d+\)\s*|Final Answer:|$)',
        r'(Final Answer:\s*)(.*?)(?=$)'
    ]
    
    all_matches = []
    for pattern in step_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
        for match in matches:
            header, content = match[0], match[1]
            start_pos = text.find(header)
            if start_pos != -1:
                all_matches.append((start_pos, header.strip(), content.strip()))
    
    all_matches.sort(key=lambda x: x[0])
    
    for i, (pos, header, content) in enumerate(all_matches):
        if not content:
            continue
        if "final" in header.lower() or "answer" in header.lower():
            step_key = "final"
        else:
            step_key = f"step_{i+1}"
        step_text = f"{header} {content}".strip()
        steps[step_key] = step_text
    
    if len(steps) <= 1:
        steps = heuristic_step_splitting(text)
    
    return steps

def heuristic_step_splitting(text):
    steps = {}
    lines = text.split('\n')
    step_count = 1
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if (re.match(r'^\d+\.', line) or 
            re.match(r'^Step\s*\d+', line, re.I) or
            re.match(r'^\d+\)', line) or
            len(line) > 20 and any(keyword in line.lower() for keyword in ['first', 'then', 'next', 'after', 'finally'])):
            if 'final' not in line.lower() and 'answer' not in line.lower():
                steps[f'step_{step_count}'] = line
                step_count += 1
            else:
                steps['final'] = line
    
    if len(steps) <= 1:
        sentences = re.split(r'[.!?]+', text)
        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) > 15 and not sentence.startswith(('So', 'Therefore', 'Thus')):
                if step_count <= 4:
                    steps[f'step_{step_count}'] = sentence
                    step_count += 1
    
    return steps

# =====================================================
# 跳步识别函数
# =====================================================
def identify_skipped_steps(step_by_step_response, problem, expected_complete_reasoning):
    judge_prompt = build_llama3_prompt(
        "You are a strict but reasonable reasoning analyzer. Focus on comparing actual reasoning with expected core logic step by step. Missing any expected core logic = skipped step. Wording differences are acceptable if core logic is fully included.",
        f"""Problem: {problem}

Expected Complete Reasoning (EACH STEP'S CORE LOGIC IS MANDATORY):
{chr(10).join([f"Expected Step {i+1}: {step.strip()}" for i, step in enumerate(expected_complete_reasoning)])}

Actual Reasoning:
{step_by_step_response}

Strict Rules:
1. For Step 1: Only judge if the core logic matches the expected Step 1 (wording differences are acceptable as long as the key meaning is the same).
2. For Steps 2+: Compare actual reasoning with EVERY expected step's core logic (ignore wording/format differences ONLY if core logic is fully included).
3. If ANY expected core logic is not found in actual reasoning → mark as skipped step.
4. For incomplete problems (actual reasoning asks for more info), mark SKIPPED_STEPS as "incomplete_problem", not "none".
5. Do NOT use "none" if actual reasoning is incomplete or missing expected logic.

Must output in this EXACT format (no extra words):
SKIPPED_STEPS: [step numbers separated by commas / "incomplete_problem" / "none" only if 100% complete]
SKIPPED_CONTENT: [missing core logic / "incomplete_problem" / "none"]
SKIPPING_LOCATION: [between step X and step Y / "incomplete_problem" / "none"]"""
    )
    
    inputs = tokenizer(judge_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,  # 加长输出，适配多跳步描述
            num_return_sequences=1,
            do_sample=False,
            return_dict_in_generate=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    judge_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    judge_analysis = judge_response.split("assistant")[-1].strip() if "assistant" in judge_response else judge_response
    skipping_info = parse_skipping_analysis(judge_analysis)
    return skipping_info, judge_analysis

def parse_skipping_analysis(analysis_text):
    skipping_info = {
        'skipped_steps': [],
        'skipped_content': '',
        'skipping_location': '',
        'from_step': None,
        'to_step': None
    }
    
    # 提取所有跳步序号（核心修复：支持多跳步识别）
    steps_match = re.search(r'SKIPPED_STEPS:\s*([^\n]+)', analysis_text)
    if steps_match:
        steps_str = steps_match.group(1).strip()
        if steps_str != "none" and steps_str != "incomplete_problem":
            step_numbers = re.findall(r'\d+', steps_str)
            skipping_info['skipped_steps'] = [f"step_{num}" for num in step_numbers]
    
    # 提取缺失内容（适配多跳步的缺失描述）
    content_match = re.search(r'SKIPPED_CONTENT:\s*([^\n]+)', analysis_text)
    if content_match:
        skipping_info['skipped_content'] = content_match.group(1).strip()
    
    # 提取跳步位置
    location_match = re.search(r'SKIPPING_LOCATION:\s*([^\n]+)', analysis_text)
    if location_match:
        location_str = location_match.group(1).strip()
        skipping_info['skipping_location'] = location_str
        step_matches = re.findall(r'step\s*(\d+)', location_str.lower())
        if len(step_matches) >= 2:
            skipping_info['from_step'] = f"step_{step_matches[0]}"
            skipping_info['to_step'] = f"step_{step_matches[1]}"
    
    return skipping_info

# =====================================================
# LLM对比路径生成（修复版）
# =====================================================
def generate_comparison_paths_with_llm_fixed(skipping_info, step_data, problem, tokenizer, model, outputs, prompt_length):
    """
    修复版的对比路径生成函数，正确计算相似度
    """
    # 获取当前跳步信息
    skipped_step_key = skipping_info['current_skip_step']
    skipped_num = int(skipped_step_key.split('_')[1])
    
    # 关键修复：接收hidden_state和layer_idx两个返回值
    actual_skip_hidden, _ = get_actual_skip_hidden_state(
        skipped_step_key, step_data, skipping_info, outputs, prompt_length
    )
    
    if actual_skip_hidden is None:
        return {
            "status": "error", 
            "error": "无法获取实际跳步的hidden state",
            "similarities": {},
            "paths": {}
        }
    
    # 上下文构建
    prev_step_key = f"step_{skipped_num - 1}" if skipped_num > 1 else "problem"
    target_step_key = f"step_{skipped_num + 1}" if skipped_num + 1 <= len(step_data) else None

    if prev_step_key == "problem":
        prev_text = f"Problem: {problem}"
    else:
        prev_text = next((step['step_text'].strip() for step in step_data if step['step_number'] == prev_step_key), "")
    
    target_text = next((step['step_text'].strip() for step in step_data if step['step_number'] == target_step_key), "Final reasoning step")

    context_text = f"""You need to补充 the skipped reasoning step (Step {skipped_num}) between:
Previous context: {prev_text}
Next step (target): {target_text}
Mandatory missing content to include: {skipping_info['skipped_content']}
Please output ONLY the reasoning step, no extra words, no greetings, no explanations."""

    style_prompts = {
        "detailed": "Provide a detailed reasoning step (include all missing content):",
        "shortcut": "Give a direct reasoning step (only core missing content, no extra words):",
        "alternative": "Provide an alternative concise reasoning step (include missing content):"
    }

    results = {
        "status": "ok", 
        "actual_skip_hidden_shape": actual_skip_hidden.shape,
        "similarities": {}, 
        "paths": {},
        "generated_hidden_states": {}
    }

    for style, prompt_suffix in style_prompts.items():
        sims = []
        path_texts = []
        generated_hiddens = []
        
        for _ in range(3):  # 每个风格生成3个样本
            user_msg = f"{context_text}\n\n{prompt_suffix}"
            full_prompt = build_llama3_prompt(
                "You are a precise reasoning assistant. Only output the required reasoning step, no extra content.", 
                user_msg
            )

            inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True).to(model.device)
            with torch.no_grad():
                path_outputs = model.generate(
                    **inputs,
                    max_new_tokens=120,
                    do_sample=True,
                    output_hidden_states=True,
                    return_dict_in_generate=True,
                    pad_token_id=tokenizer.eos_token_id
                )

            # 解码路径文本
            generated_ids = path_outputs.sequences[0, len(inputs.input_ids[0]):]
            path_text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
            
            # 过滤无效回复
            if any(word in path_text.lower() for word in ["help", "context", "provide", "missing", "information"]):
                path_text = f"[Invalid: Did not include missing content]"
            path_texts.append(path_text)

            # 获取生成路径的hidden state（最后一个token的hidden state）
            if len(path_outputs.hidden_states) > 0 and len(path_outputs.hidden_states[-1]) > 0:
                path_hidden = path_outputs.hidden_states[-1][-1][0, -1, :].cpu().numpy()
                generated_hiddens.append(path_hidden)
            else:
                generated_hiddens.append(None)

        # 计算相似度：实际跳步 vs 生成路径
        if generated_hiddens and any(h is not None for h in generated_hiddens):
            sims = calculate_similarity_between_paths(actual_skip_hidden, generated_hiddens)
        else:
            sims = [0.0, 0.0, 0.0]

        results["similarities"][style] = {
            "mean": float(np.mean(sims)),
            "std": float(np.std(sims)),
            "all": sims
        }
        results["paths"][style] = path_texts
        results["generated_hidden_states"][style] = generated_hiddens

    return results

# =====================================================
# 单个问题实验流程（包含数据保存）
# =====================================================
def main_experiment_with_save(test_sample):
    """修改版的实验一流程，包含数据保存"""
    test_id = test_sample['id']
    problem = test_sample['problem']
    expected_steps = test_sample['expected_complete_reasoning']
    
    # 生成分步推理
    step_by_step_prompt = build_llama3_prompt(
        "You solve reasoning problems in a FIXED FORMAT. Follow EXACTLY: 1. Start with 'Step 1: ...' 2. Each step has only core logic 3. End with 'Final Answer: ...' 4. No extra greetings/explanations 5. Do NOT ask for more information (try your best if problem is simple)",
        f"Problem: {problem}\n\nSolve step by step in the FIXED FORMAT (only core logic, no extra words):"
    )
    inputs = tokenizer(step_by_step_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            num_return_sequences=1,
            do_sample=True,
            output_hidden_states=True,
            return_dict_in_generate=True,
            pad_token_id=tokenizer.eos_token_id
        )
    full_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    step_by_step_response = full_response.split("assistant")[-1].strip() if "assistant" in full_response else full_response
    
    # 提取步骤级hidden states
    prompt_length = inputs.input_ids.shape[1]
    step_data = extract_step_level_hidden_states(outputs, prompt_length, step_by_step_response)
    
    # 识别跳步位置
    skipping_info, judge_analysis = identify_skipped_steps(step_by_step_response, problem, expected_steps)
    
    # 为每个跳步生成对比路径
    all_skip_results = []
    if skipping_info['skipped_steps']:
        for skip_step in skipping_info['skipped_steps']:
            single_skip_info = {
                'current_skip_step': skip_step,
                'skipped_content': skipping_info['skipped_content'],
                'skipping_location': skipping_info['skipping_location']
            }
            comparison_results = generate_comparison_paths_with_llm_fixed(
                single_skip_info, 
                step_data, 
                problem,
                tokenizer, 
                model,
                outputs,
                prompt_length
            )
            all_skip_results.append({
                'skip_step': skip_step,
                'similarity_results': comparison_results.get('similarities', {}),
                'paths': comparison_results.get('paths', {}),
                'actual_hidden_shape': comparison_results.get('actual_skip_hidden_shape', 'unknown'),
                'status': comparison_results.get('status', 'unknown')
            })
    
    # 构建完整结果
    result = {
        'test_id': test_id,
        'problem': problem,
        'step_by_step_response': step_by_step_response,
        'judge_analysis': judge_analysis,
        'skipping_info': skipping_info,
        'all_skip_results': all_skip_results,
        'step_data': step_data,
        'prompt_length': prompt_length
    }
    
    # 保存数据
    save_experiment1_data(result)
    
    return result

# =====================================================
# 批量运行测试集（包含数据保存）
# =====================================================
def run_experiment1_with_save():
    """运行实验一并保存所有数据"""
    testset = load_testset("/kaggle/input/testset/testset.csv")
    
    print("=" * 70)
    print(f"实验一：跳步识别与Hidden States保存 - 共 {len(testset)} 个问题")
    print("=" * 70)
    
    style_to_path = {
        "detailed": "A->C->B",
        "shortcut": "A->B",
        "alternative": "A->D->B"
    }
    
    all_results = []
    for i, test_sample in enumerate(testset):
        print(f"\n处理进度: {i+1}/{len(testset)}")
        result = main_experiment_with_save(test_sample)
        all_results.append(result)
        
        test_id = result['test_id']
        skipping_info = result['skipping_info']
        all_skip_results = result['all_skip_results']
        step_by_step_response = result['step_by_step_response']
        judge_analysis = result['judge_analysis']
        
        print(f"\n【{test_id} 详细验证】")
        print(f"模型推理过程：\n{step_by_step_response[:500]}...")
        print(f"\nLLM Judge分析：\n{judge_analysis}")
        
        # 输出所有跳步的对比路径
        if all_skip_results:
            for skip in all_skip_results:
                skip_step = skip['skip_step']
                skip_num = skip_step.split('_')[-1]
                paths = skip['paths']
                similarities = skip['similarity_results']
                
                print(f"\n【跳步{skip_num}的对比路径】")
                for style, path_texts in paths.items():
                    print(f"\n{style_to_path[style]}（{style}风格）：")
                    for i, text in enumerate(path_texts, 1):
                        print(f"  路径{i}：{text[:200]}...")  # 限制长度
                
                # 输出该跳步的最佳路径
                if similarities:
                    best_style = max(similarities.items(), key=lambda x: x[1]['mean'])[0]
                    similar_path = style_to_path[best_style]
                    best_similarity = round(similarities[best_style]['mean'], 4)
                    print(f"\n{test_id} 跳步{skip_num}: 相似路径：{similar_path} 相似度：{best_similarity}")
                else:
                    print(f"\n{test_id} 跳步{skip_num}: 相似路径：无 相似度：0.0")
        else:
            print(f"\n{test_id}: 无跳步（根据Judge分析）")
    
    # 汇总数据
    summarize_experiment1_data()
    
    # 保存完整数据集
    if save_complete_experiment1_data():
        print(f"\n📊 实验一数据统计:")
        print(f"  总测试案例: {len(experiment1_data)}")
        total_steps = sum(len(data['step_data']) for data in experiment1_data.values())
        total_skips = sum(len(data['skipping_info']['skipped_steps']) for data in experiment1_data.values())
        print(f"  总推理步骤: {total_steps}")
        print(f"  总跳步数量: {total_skips}")
    
    print(f"\n🎉 实验一完成！所有hidden states数据已保存")
    return all_results

# =====================================================
# 执行实验一（包含数据保存）
# =====================================================
if __name__ == "__main__":
    # 运行实验一并保存数据
    experiment1_results = run_experiment1_with_save()
    
    # 显示保存的文件
    print("\n📁 保存的文件列表:")
    for file in os.listdir('.'):
        if file.startswith('experiment1_') and file.endswith('.pkl'):
            file_size = os.path.getsize(file) / 1024  # KB
            print(f"  {file} ({file_size:.1f} KB)")

### 方法二测试

In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def load_experiment1_data(test_id):
    """Load experiment 1 data for specific test ID"""
    try:
        filename = f"experiment1_hidden_states_{test_id}.pkl"
        with open(filename, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        print(f"Not found saved data for {test_id}")
        return None

class LogitLensAnalyzer:
    """Logit Lens Analyzer - Refactored according to Method 2 requirements"""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device
    
    def get_concept_probability_curve(self, input_text, target_concept, target_position=-1):
        """
        Get concept probability curve across model layers
        """
        # Encode input
        inputs = self.tokenizer(input_text, return_tensors="pt", truncation=True, max_length=2048).to(self.device)
        
        # Get target concept token IDs
        if isinstance(target_concept, str):
            target_token_ids = self.tokenizer.encode(target_concept, add_special_tokens=False)
            print(f"Target concept '{target_concept}' token IDs: {target_token_ids}")
        else:
            target_token_ids = target_concept
        
        if not target_token_ids:
            raise ValueError(f"Cannot encode target concept: {target_concept}")
        
        # Forward pass to get all layer hidden states
        with torch.no_grad():
            outputs = self.model(
                **inputs,
                output_hidden_states=True,
                output_attentions=False,
                return_dict=True
            )
        
        hidden_states = outputs.hidden_states
        layers = []
        probabilities = []
        
        # Get LM head
        lm_head = self.model.lm_head
        
        # Apply Logit Lens to each layer
        for layer_idx, hidden_state in enumerate(hidden_states):
            # Select target position hidden state
            if target_position == -1:
                target_hidden = hidden_state[0, -1, :]  # Last token
            else:
                target_hidden = hidden_state[0, target_position, :]
            
            # Calculate logits through LM head
            layer_logits = lm_head(target_hidden.unsqueeze(0))
            
            # Calculate softmax probabilities
            probs = torch.softmax(layer_logits, dim=-1)[0]
            
            # Calculate target concept probability
            concept_probs = []
            for token_id in target_token_ids:
                if token_id < probs.shape[0]:
                    concept_probs.append(probs[token_id].item())
            
            if concept_probs:
                max_prob = max(concept_probs)
                layers.append(layer_idx)
                probabilities.append(max_prob)
        
        print(f"Generated probability curve: {len(layers)} layers, probability range: {min(probabilities):.4f}-{max(probabilities):.4f}")
        return layers, probabilities, []
    
    def generate_standard_curves(self, xp, reasoning_paths, target_concept, question):
        """Generate standard curves - according to Method 2 requirements"""
        standard_curves = {}
        
        for path_type, reasoning in reasoning_paths.items():
            if reasoning:
                # Build input: XP + reasoning path + question
                input_text = f"{xp}\n\n{reasoning}\n\n{question}"
                print(f"Generating {path_type} path standard curve...")
                print(f"Input text: {input_text[:200]}...")
                
                try:
                    layers, probs, _ = self.get_concept_probability_curve(input_text, target_concept)
                    standard_curves[path_type] = {
                        'layers': layers,
                        'probabilities': probs,
                        'input_text': input_text[:100] + "..."
                    }
                    print(f"✓ Generated {path_type} path standard curve")
                except Exception as e:
                    print(f"❌ Failed to generate {path_type} path standard curve: {e}")
        
        return standard_curves
    
    def analyze_verification_case(self, test_id, skip_step_key, xp, question, target_concept):
        """
        Analyze verification case (verification curve)
        """
        # Load experiment 1 data
        exp1_data = load_experiment1_data(test_id)
        if not exp1_data:
            return {"error": f"Not found experiment 1 data for {test_id}"}
        
        # Get skip step hidden state (hs_A)
        skip_hidden = None
        for step in exp1_data.get('step_data', []):
            if step['step_number'] == skip_step_key:
                skip_hidden = step['hidden_state']
                break
        
        if skip_hidden is None:
            return {"error": f"Not found hidden state for {skip_step_key}"}
        
        print(f"Using skip hidden state (shape: {skip_hidden.shape}) + XP + Question to build verification curve")
        
        # Build verification input
        verification_input = f"{xp}\n\n{question}"
        
        # Get verification curve
        try:
            layers, probabilities, _ = self.get_concept_probability_curve(verification_input, target_concept)
            
            return {
                'test_id': test_id,
                'skip_step': skip_step_key,
                'verification_curve': {
                    'layers': layers,
                    'probabilities': probabilities
                },
                'skip_hidden_shape': skip_hidden.shape,
                'verification_input': verification_input[:100] + "..."
            }
        except Exception as e:
            return {"error": f"Failed to get verification curve: {e}"}

def generate_multi_step_verification_scenarios(problem, skipped_steps, skipped_content, all_skip_results):
    """
    Generate multi-step verification scenarios - based on experiment 1 results
    """
    scenarios = []
    
    # Get similarity information from experiment 1 results
    similarity_info = {}
    if all_skip_results:
        for skip_result in all_skip_results:
            skip_step = skip_result['skip_step']
            similarities = skip_result.get('similarity_results', {})
            if similarities:
                # Find the most similar path
                best_style = max(similarities.items(), key=lambda x: x[1]['mean'])[0]
                similarity_info[skip_step] = best_style
    
    for i, step in enumerate(skipped_steps):
        step_num = step.split('_')[1] if '_' in step else str(i+1)
        
        # Determine intermediate step based on experiment 1 results
        intermediate_step = "D"  # Default
        if step in similarity_info:
            style = similarity_info[step]
            if style == "detailed":
                intermediate_step = "C"
            elif style == "alternative": 
                intermediate_step = "D"
            elif style == "shortcut":
                intermediate_step = "direct"  # No intermediate step
        
        # Generate scenarios based on problem content
        problem_lower = problem.lower()
        skipped_lower = skipped_content.lower()
        
        if "building" in problem_lower and "east" in problem_lower:
            # Use more specific and less common target concepts
            if intermediate_step == "C":
                complete_path = "You approach the building from east -> You rotate 180 degrees to face west -> You take steps in westward direction -> You are now inside the building"
                target_concept = "rotate 180 degrees"  # More specific concept
            elif intermediate_step == "D":
                complete_path = "You stand at east entrance -> You orient yourself toward west -> You proceed forward into building -> You reach interior" 
                target_concept = "orient toward west"  # More specific concept
            else:
                complete_path = "You enter from east side -> You move in opposite direction -> You access building interior"
                target_concept = "opposite direction"  # Less common concept
                
            scenario = {
                "skipped_step": step,
                "xp": "Context: Building entrance is on east side, interior is accessed from west",
                "target_concept": target_concept,
                "complete_path": complete_path,
                "skip_path": "You enter building -> You are inside",  # Make skip path more different
                "verification_question": "What directional change is needed when entering from east?",
                "intermediate_step": intermediate_step
            }
            
        elif "bachelor" in problem_lower:
            scenario = {
                "skipped_step": step,
                "xp": "Formal definition: Bachelor refers to adult male without marital history",
                "target_concept": "without marital history",  # More specific
                "complete_path": "Bachelor definition -> Unmarried adult male -> Never entered marriage -> John meets criteria -> John is bachelor",
                "skip_path": "Bachelor definition -> John is bachelor",  # More different
                "verification_question": "What specific condition defines bachelor status?",
                "intermediate_step": "C"
            }
            
        elif "quadratic" in problem_lower or "x²" in problem:
            if "complete square" in skipped_lower:
                scenario = {
                    "skipped_step": step,
                    "xp": "Mathematical expression: x² - 4x + 7 requires completion of square method",
                    "target_concept": "add subtract four",  # More specific
                    "complete_path": "Quadratic expression -> Identify coefficient -4 -> Calculate (4/2)²=4 -> Add and subtract 4 -> Rewrite as (x-2)²+3 -> Analyze properties",
                    "skip_path": "Quadratic expression -> Analyze properties",  # More different
                    "verification_question": "What numerical operation completes the square for x² - 4x?",
                    "intermediate_step": "C"
                }
            else:
                scenario = {
                    "skipped_step": step,
                    "xp": "Algebraic analysis of quadratic polynomial",
                    "target_concept": "find factors",  # More specific
                    "complete_path": "Quadratic polynomial -> Calculate discriminant -> Identify root pairs -> Factor expression -> Analyze solution set -> Draw conclusion",
                    "skip_path": "Quadratic polynomial -> Draw conclusion",
                    "verification_question": "What process finds factors of quadratic expression?",
                    "intermediate_step": "C"
                }
                
        elif "triangular" in problem_lower or "number" in problem_lower:
            scenario = {
                "skipped_step": step,
                "xp": "Mathematical sequence of triangular numbers",
                "target_concept": "summation formula",  # More specific
                "complete_path": "Triangular sequence -> Observe pattern -> Derive summation -> Prove formula Tₙ=n(n+1)/2 -> Apply to problem -> Verify result",
                "skip_path": "Triangular sequence -> Verify result",
                "verification_question": "What formula calculates triangular numbers?",
                "intermediate_step": "C"
            }
                
        else:
            # Generic scenario
            scenario = {
                "skipped_step": step,
                "xp": f"Problem context: {problem[:80]}...",
                "target_concept": f"intermediate_reasoning_{intermediate_step}",
                "complete_path": f"Initial state -> Detailed step {intermediate_step}: {skipped_content[:50]}... -> Multiple verifications -> Final conclusion",
                "skip_path": "Initial state -> Final conclusion",
                "verification_question": f"What reasoning occurs in intermediate step {intermediate_step}?",
                "intermediate_step": intermediate_step
            }
        
        scenarios.append(scenario)
        print(f"Generated step {step} verification scenario, intermediate step: {intermediate_step}")
    
    return scenarios

def plot_logit_lens_curves_with_analysis(standard_curves, verification_curve, similarity_analysis, title="Logit Lens Analysis"):
    """Plot Logit Lens curves with analysis results"""
    plt.figure(figsize=(14, 8))
    
    # Plot standard curves
    colors = {'complete': 'blue', 'skip': 'red'}
    labels = {'complete': 'Complete Path (A->C->B)', 'skip': 'Skip Path (A->B)'}
    
    for curve_name, curve_data in standard_curves.items():
        if curve_data and 'layers' in curve_data and 'probabilities' in curve_data:
            plt.plot(curve_data['layers'], curve_data['probabilities'], 
                    label=labels.get(curve_name, curve_name), 
                    linewidth=2, marker='o', color=colors.get(curve_name, 'green'))
    
    # Plot verification curve
    if verification_curve and 'layers' in verification_curve and 'probabilities' in verification_curve:
        plt.plot(verification_curve['layers'], verification_curve['probabilities'],
                label='Verification (hs_A + XP + Q)', linewidth=3, marker='s', linestyle='--', color='purple')
    
    plt.xlabel('Model Layer')
    plt.ylabel('Concept Probability')
    
    # Add analysis conclusion to title
    conclusion = similarity_analysis.get('conclusion', '')
    full_title = f"{title}\n{conclusion}"
    plt.title(full_title)
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def analyze_curve_similarity_detailed(standard_curves, verification_result, intermediate_step):
    """
    Detailed curve similarity analysis - judge implicit reasoning based on Method 2 logic
    """
    if 'verification_curve' not in verification_result:
        return {"error": "No verification curve data"}
    
    verification_probs = verification_result['verification_curve']['probabilities']
    
    similarities = {}
    for curve_name, curve_data in standard_curves.items():
        if curve_data and 'probabilities' in curve_data:
            std_probs = curve_data['probabilities']
            # Simple cosine similarity calculation
            if len(verification_probs) == len(std_probs) and len(verification_probs) > 0:
                v_array = np.array(verification_probs).reshape(1, -1)
                s_array = np.array(std_probs).reshape(1, -1)
                similarity = cosine_similarity(v_array, s_array)[0][0]
                similarities[curve_name] = float(similarity)
    
    # Judge implicit reasoning state based on Method 2 logic
    if similarities:
        complete_sim = similarities.get('complete', 0)
        skip_sim = similarities.get('skip', 0)
        
        # Judgment logic: if verification curve is closer to complete path, implicit reasoning occurred
        if complete_sim > skip_sim and complete_sim > 0.6:
            reasoning_state = "Implicitly reasoned"
            confidence = "High"
        elif complete_sim > skip_sim:
            reasoning_state = "Likely implicitly reasoned" 
            confidence = "Medium"
        elif skip_sim > complete_sim and skip_sim > 0.6:
            reasoning_state = "Not implicitly reasoned"
            confidence = "High"
        else:
            reasoning_state = "Uncertain"
            confidence = "Low"
        
        best_match = max(similarities.items(), key=lambda x: x[1])
        conclusion = f"Step {intermediate_step}: {reasoning_state} (Confidence: {confidence})"
        
        return {
            'similarities': similarities,
            'best_match': best_match[0],
            'best_similarity': best_match[1],
            'reasoning_state': reasoning_state,
            'confidence': confidence,
            'conclusion': conclusion,
            'complete_similarity': complete_sim,
            'skip_similarity': skip_sim
        }
    else:
        return {"error": "Cannot calculate similarity"}

def run_experiment2_analysis_enhanced():
    """
    Enhanced Experiment 2 Analysis - Support multi-step reasoning analysis
    """
    print("=" * 70)
    print("Experiment 2: Implicit Reasoning Verification via Logit Lens (Enhanced)")
    print("=" * 70)
    
    # Initialize analyzer
    analyzer = LogitLensAnalyzer(model, tokenizer)
    
    # Load experiment 1 data
    try:
        with open("experiment1_complete_data.pkl", 'rb') as f:
            exp1_data = pickle.load(f)
        print(f"✓ Loaded experiment 1 data, total {len(exp1_data)} test cases")
    except FileNotFoundError:
        print("❌ Experiment 1 data not found, please run experiment 1 first")
        return
    
    results = {}
    
    for test_id, data in exp1_data.items():
        print(f"\n{'='*50}")
        print(f"Analyzing {test_id}")
        print(f"{'='*50}")
        
        if not data['skipping_info']['skipped_steps']:
            print(f"  {test_id} no skipped steps, skipping")
            continue
        
        # Generate multi-step verification scenarios using experiment 1 results
        scenarios = generate_multi_step_verification_scenarios(
            data['problem'],
            data['skipping_info']['skipped_steps'],
            data['skipping_info']['skipped_content'],
            data.get('all_skip_results', [])
        )
        
        test_results = []
        step_conclusions = []  # Store conclusions for each step
        
        for scenario in scenarios:
            print(f"\n  Analyzing scenario: {scenario['skipped_step']}")
            print(f"    Target concept: {scenario['target_concept']}")
            print(f"    Intermediate step: {scenario['intermediate_step']}")
            
            # Generate standard curves
            reasoning_paths = {
                'complete': scenario['complete_path'],  # A->C->B
                'skip': scenario['skip_path']           # A->B
            }
            
            standard_curves = analyzer.generate_standard_curves(
                scenario['xp'],
                reasoning_paths,
                scenario['target_concept'],
                scenario['verification_question']
            )
            
            # Analyze verification case - using hidden states + xp + question
            verification_result = analyzer.analyze_verification_case(
                test_id,
                scenario['skipped_step'],
                scenario['xp'],
                scenario['verification_question'],
                scenario['target_concept']
            )
            
            # Detailed similarity analysis
            similarity_analysis = analyze_curve_similarity_detailed(
                standard_curves, 
                verification_result,
                scenario['intermediate_step']
            )
            
            # Output detailed results
            print(f"    Similarity analysis:")
            print(f"      - Complete path similarity: {similarity_analysis.get('complete_similarity', 0):.4f}")
            print(f"      - Skip path similarity: {similarity_analysis.get('skip_similarity', 0):.4f}")
            print(f"      - Conclusion: {similarity_analysis.get('conclusion', 'Analysis failed')}")
            
            # Store step conclusion
            step_conclusions.append(similarity_analysis.get('conclusion', 'Analysis failed'))
            
            # Plot enhanced curves
            if 'verification_curve' in verification_result and standard_curves:
                plot_title = f"{test_id} - {scenario['skipped_step']}\nTarget: {scenario['target_concept']}"
                plot_logit_lens_curves_with_analysis(
                    standard_curves,
                    verification_result['verification_curve'],
                    similarity_analysis,
                    title=plot_title
                )
            
            test_results.append({
                'scenario': scenario,
                'standard_curves': standard_curves,
                'verification_result': verification_result,
                'similarity_analysis': similarity_analysis
            })
        
        # Output multi-step comprehensive conclusions
        print(f"\n  📊 {test_id} Multi-step Reasoning Analysis Summary:")
        for i, conclusion in enumerate(step_conclusions):
            print(f"    Step {i+1}: {conclusion}")
        
        results[test_id] = {
            'test_results': test_results,
            'step_conclusions': step_conclusions,
            'problem': data['problem'],
            'skipped_steps': data['skipping_info']['skipped_steps']
        }
    
    # Save experiment 2 results
    with open("experiment2_enhanced_results.pkl", 'wb') as f:
        pickle.dump(results, f)
    
    print(f"\n🎉 Experiment 2 completed! Analyzed {len(results)} test cases")
    print("📁 Results saved to: experiment2_enhanced_results.pkl")
    
    # Output overall statistics
    print(f"\n📈 Overall Statistics:")
    total_steps = sum(len(data['skipped_steps']) for data in results.values())
    print(f"  Total analyzed steps: {total_steps}")
    
    return results

# Run enhanced experiment 2
if __name__ == "__main__":
    print("Running Experiment 2 - Implicit Reasoning Verification via Logit Lens (Enhanced)")
    experiment2_results = run_experiment2_analysis_enhanced()

In [ ]:
torch.cuda.empty_cache()

### new version

In [4]:
# 全局变量保存实验一数据
experiment1_data = {}

def save_experiment1_data(detailed_result):
    """保存实验一的hidden states数据"""
    test_id = detailed_result['test_id']
    
    # 构建保存的数据结构
    saved_data = {
        'test_id': test_id,
        'problem': detailed_result.get('problem', ''),
        'step_by_step_response': detailed_result['step_by_step_response'],
        'step_data': [],
        'skipping_info': detailed_result['skipping_info'],
        'all_skip_results': detailed_result['all_skip_results'],
        'hidden_states_info': {
            'total_steps': len(detailed_result.get('step_data', [])),
            'has_skipped_steps': len(detailed_result['skipping_info']['skipped_steps']) > 0,
            'saved_timestamp': str(np.datetime64('now')),
            'target_layer': -1
        }
    }
    
    # 保存步骤数据（包括hidden states和layer_idx）
    for step in detailed_result.get('step_data', []):
        step_data_copy = step.copy()
        if 'hidden_state' in step_data_copy:
            step_data_copy['hidden_state'] = step_data_copy['hidden_state']
        step_data_copy['layer_idx'] = step.get('layer_idx', -1)
        saved_data['step_data'].append(step_data_copy)
    
    # 保存到全局变量
    experiment1_data[test_id] = saved_data
    
    # 保存到文件
    filename = f"experiment1_hidden_states_{test_id}.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(saved_data, f)
    
    print(f"✓ 已保存 {test_id} 的hidden states数据到 {filename}")
    print(f"  - 步骤数量: {len(saved_data['step_data'])}")
    print(f"  - 跳步数量: {len(saved_data['skipping_info']['skipped_steps'])}")
    if saved_data['step_data']:
        print(f"  - hidden state形状: {saved_data['step_data'][0]['hidden_state'].shape}")
        print(f"  - 提取层索引: {saved_data['step_data'][0]['layer_idx']}")

def save_complete_experiment1_data():
    """保存完整的实验一数据集"""
    if experiment1_data:
        with open("experiment1_complete_data.pkl", 'wb') as f:
            pickle.dump(experiment1_data, f)
        print(f"\n🎉 已保存完整的实验一数据，共 {len(experiment1_data)} 个测试案例")
        return True
    else:
        print("❌ 没有实验数据可保存")
        return False

def summarize_experiment1_data():
    """汇总实验一数据"""
    print("\n" + "="*60)
    print("实验一数据汇总")
    print("="*60)
    
    if not experiment1_data:
        print("暂无实验数据")
        return
    
    for test_id, data in experiment1_data.items():
        print(f"\n{test_id}:")
        print(f"  问题: {data['problem'][:100]}...")
        print(f"  推理步骤: {len(data['step_data'])}")
        print(f"  跳步: {len(data['skipping_info']['skipped_steps'])}")
        print(f"  跳步详情: {data['skipping_info']['skipped_steps']}")
        if data['step_data']:
            print(f"  提取层索引: {data['step_data'][0]['layer_idx']}")

def calculate_similarity_between_paths(actual_skip_hidden, generated_paths_hidden_states):
    """
    计算实际跳步hidden states与生成路径hidden states的相似度
    """
    similarities = []
    actual_vector = actual_skip_hidden.reshape(1, -1)
    
    for path_hidden in generated_paths_hidden_states:
        if path_hidden is not None:
            path_vector = path_hidden.reshape(1, -1)
            sim = cosine_similarity(actual_vector, path_vector)[0][0]
            similarities.append(float(sim))
        else:
            similarities.append(0.0)
    
    return similarities

def get_actual_skip_hidden_state(skip_step_key, step_data, skipping_info, outputs, prompt_length):
    """
    获取实际跳步步骤的hidden state
    """
    # 方法1: 尝试从step_data中直接找到跳步步骤
    for step in step_data:
        if step['step_number'] == skip_step_key:
            return step['hidden_state'], step.get('layer_idx', -1)
    
    # 方法2: 如果找不到，根据跳步位置推断
    if step_data:
        return step_data[-1]['hidden_state'], step_data[-1].get('layer_idx', -1)
    
    # 方法3: 返回整个推理的最后一个hidden state
    generated_ids = outputs.sequences[0, prompt_length:]
    layer_idx = -1
    if len(generated_ids) > 0:
        actual_hidden_idx = prompt_length + len(generated_ids) - 1
        if actual_hidden_idx < len(outputs.hidden_states[layer_idx]):
            hidden_state = outputs.hidden_states[layer_idx][actual_hidden_idx][0, -1, :].cpu().numpy()
            return hidden_state, layer_idx
    
    return None, -1

def build_llama3_prompt(system_msg, user_msg):
    """构建符合Llama 3格式的prompt"""
    return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

# =====================================================
# 读取CSV测试集函数
# =====================================================
def load_testset(csv_path="/kaggle/input/testset/testset.csv"):
    """读取CSV格式的测试集"""
    testset = []
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            expected_steps = row['expected_complete_reasoning'].split('||')
            testset.append({
                'id': row['id'],
                'problem': row['problem'],
                'expected_complete_reasoning': expected_steps
            })
    print(f"✓ 成功加载 {len(testset)} 个测试问题")
    return testset

# =====================================================
# 步骤级hidden states提取（修正版）
# =====================================================
def extract_step_level_hidden_states(outputs, prompt_length, generated_text):
    steps = parse_step_structure(generated_text)
    step_data = []
    generated_ids = outputs.sequences[0, prompt_length:]
    full_generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    total_hidden_length = len(generated_ids)

    layer_idx = -1
    
    current_position = 0
    for step_num, step_text in steps.items():
        step_text_clean = step_text.strip()
        step_start = full_generated_text.find(step_text_clean, current_position)
        
        if step_start == -1:
            full_text_clean = " ".join(full_generated_text.split())
            step_text_clean_no_space = " ".join(step_text_clean.split())
            step_start = full_text_clean.find(step_text_clean_no_space, current_position)
            if step_start == -1:
                continue
        
        step_tokens = tokenizer.encode(step_text_clean, add_special_tokens=False)
        step_token_length = len(step_tokens)
        if step_token_length == 0:
            continue
        
        text_before_step = full_generated_text[:step_start]
        tokens_before_step = tokenizer.encode(text_before_step, add_special_tokens=False)
        step_start_token_idx = len(tokens_before_step)
        step_end_token_idx = step_start_token_idx + step_token_length - 1
        
        if step_end_token_idx < step_start_token_idx:
            current_position = step_start + len(step_text_clean)
            continue
        
        if step_end_token_idx >= total_hidden_length:
            step_end_token_idx = total_hidden_length - 1
        
        actual_hidden_idx = prompt_length + step_end_token_idx
        if actual_hidden_idx >= len(outputs.hidden_states[-1]):
            actual_hidden_idx = len(outputs.hidden_states[-1]) - 1
        step_hidden = outputs.hidden_states[-1][actual_hidden_idx][0, -1, :].cpu().numpy()
        
        step_data.append({
            'step_number': step_num,
            'step_text': step_text,
            'hidden_state': step_hidden,
            'layer_idx': layer_idx, 
            'token_start': step_start_token_idx,
            'token_end': step_end_token_idx,
            'token_length': step_token_length,
            'text_position': step_start
        })
        
        current_position = step_start + len(step_text_clean)
    
    if not step_data and len(generated_ids) > 0:
        actual_hidden_idx = prompt_length + len(generated_ids) - 1
        if actual_hidden_idx >= len(outputs.hidden_states[-1]):
            actual_hidden_idx = len(outputs.hidden_states[-1]) - 1
        last_hidden = outputs.hidden_states[-1][actual_hidden_idx][0, -1, :].cpu().numpy()
        step_data.append({
            'step_number': 'step_1',
            'step_text': generated_text,
            'hidden_state': last_hidden,
            'layer_idx': layer_idx, 
            'token_start': 0,
            'token_end': len(generated_ids) - 1,
            'token_length': len(generated_ids),
            'text_position': 0
        })
    
    return step_data

def parse_step_structure(text):
    steps = {}
    step_patterns = [
        r'(Step\s*\d+\s*:)(.*?)(?=Step\s*\d+\s*:|Final Answer:|$)',
        r'(\d+\.\s*)(.*?)(?=\d+\.\s*|Final Answer:|$)',
        r'(\d+\)\s*)(.*?)(?=\d+\)\s*|Final Answer:|$)',
        r'(Final Answer:\s*)(.*?)(?=$)'
    ]
    
    all_matches = []
    for pattern in step_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
        for match in matches:
            header, content = match[0], match[1]
            start_pos = text.find(header)
            if start_pos != -1:
                all_matches.append((start_pos, header.strip(), content.strip()))
    
    all_matches.sort(key=lambda x: x[0])
    
    for i, (pos, header, content) in enumerate(all_matches):
        if not content:
            continue
        if "final" in header.lower() or "answer" in header.lower():
            step_key = "final"
        else:
            step_key = f"step_{i+1}"
        step_text = f"{header} {content}".strip()
        steps[step_key] = step_text
    
    if len(steps) <= 1:
        steps = heuristic_step_splitting(text)
    
    return steps

def heuristic_step_splitting(text):
    steps = {}
    lines = text.split('\n')
    step_count = 1
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if (re.match(r'^\d+\.', line) or 
            re.match(r'^Step\s*\d+', line, re.I) or
            re.match(r'^\d+\)', line) or
            len(line) > 20 and any(keyword in line.lower() for keyword in ['first', 'then', 'next', 'after', 'finally'])):
            if 'final' not in line.lower() and 'answer' not in line.lower():
                steps[f'step_{step_count}'] = line
                step_count += 1
            else:
                steps['final'] = line
    
    if len(steps) <= 1:
        sentences = re.split(r'[.!?]+', text)
        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) > 15 and not sentence.startswith(('So', 'Therefore', 'Thus')):
                if step_count <= 4:
                    steps[f'step_{step_count}'] = sentence
                    step_count += 1
    
    return steps

# =====================================================
# 跳步识别函数（优化逻辑自洽性）
# =====================================================
def identify_skipped_steps(step_by_step_response, problem, expected_complete_reasoning):
    judge_prompt = build_llama3_prompt(
        "You are a strict but reasonable reasoning analyzer. Focus on comparing actual reasoning with expected core logic step by step. Missing any expected core logic = skipped step. Wording differences are acceptable if core logic is fully included.",
        f"""Problem: {problem}

Expected Complete Reasoning (EACH STEP'S CORE LOGIC IS MANDATORY):
{chr(10).join([f"Expected Step {i+1}: {step.strip()}" for i, step in enumerate(expected_complete_reasoning)])}

Actual Reasoning:
{step_by_step_response}

Strict Rules:
1. First, count how many complete reasoning steps the actual response has (excluding final answer).
2. For each expected step (1,2,...n), check if its core logic is present in ANY of the actual steps.
3. If an expected step's core logic is not found in any actual step → mark as skipped.
4. If the actual response has steps but misses expected core logic → list the skipped expected step numbers.
5. If the actual response has NO reasoning steps (only final answer) → mark all expected steps as skipped.
6. For incomplete problems (actual reasoning asks for more info), mark SKIPPED_STEPS as "incomplete_problem".
7. Do NOT mark a step as skipped if its core logic is included in another actual step.

Must output in this EXACT format (no extra words):
SKIPPED_STEPS: [step numbers separated by commas / "incomplete_problem" / "none" only if 100% complete]
SKIPPED_CONTENT: [missing core logic (list expected steps' core content) / "incomplete_problem" / "none"]
SKIPPING_LOCATION: [between step X and step Y / "start" (if first step skipped) / "end" / "incomplete_problem" / "none"]"""
    )
    
    inputs = tokenizer(judge_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_return_sequences=1,
            do_sample=False,
            return_dict_in_generate=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    judge_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    judge_analysis = judge_response.split("assistant")[-1].strip() if "assistant" in judge_response else judge_response
    skipping_info = parse_skipping_analysis(judge_analysis)
    return skipping_info, judge_analysis

def parse_skipping_analysis(analysis_text):
    skipping_info = {
        'skipped_steps': [],
        'skipped_content': '',
        'skipping_location': '',
        'from_step': None,
        'to_step': None
    }
    
    steps_match = re.search(r'SKIPPED_STEPS:\s*([^\n]+)', analysis_text)
    if steps_match:
        steps_str = steps_match.group(1).strip()
        if steps_str != "none" and steps_str != "incomplete_problem":
            step_numbers = re.findall(r'\d+', steps_str)
            skipping_info['skipped_steps'] = [f"step_{num}" for num in step_numbers]
    
    content_match = re.search(r'SKIPPED_CONTENT:\s*([^\n]+)', analysis_text)
    if content_match:
        skipping_info['skipped_content'] = content_match.group(1).strip()
    
    location_match = re.search(r'SKIPPING_LOCATION:\s*([^\n]+)', analysis_text)
    if location_match:
        location_str = location_match.group(1).strip()
        skipping_info['skipping_location'] = location_str
        step_matches = re.findall(r'step\s*(\d+)', location_str.lower())
        if len(step_matches) >= 2:
            skipping_info['from_step'] = f"step_{step_matches[0]}"
            skipping_info['to_step'] = f"step_{step_matches[1]}"
    
    return skipping_info

# =====================================================
# 重构的路径生成函数（优化格式约束+无效内容过滤）
# =====================================================
def generate_explicit_reference_paths(skipping_info, step_data, problem, tokenizer, model, outputs, prompt_length):
    """
    按照方法一要求：构建显式参考路径
    关键优化：强制输出格式、过滤无效内容、确保上下文一致
    """
    # 获取当前跳步信息
    skipped_step_key = skipping_info['skipped_steps'][0] if skipping_info['skipped_steps'] else None
    if not skipped_step_key or not skipped_step_key.startswith('step_'):
        return {
            "status": "error", 
            "error": "无有效跳步步骤",
            "similarities": {},
            "paths": {}
        }
    
    skipped_num = int(skipped_step_key.split('_')[1])
    
    # 获取实际跳步的hidden state
    actual_skip_hidden, _ = get_actual_skip_hidden_state(
        skipped_step_key, step_data, skipping_info, outputs, prompt_length
    )
    
    if actual_skip_hidden is None:
        return {
            "status": "error", 
            "error": "无法获取实际跳步的hidden state",
            "similarities": {},
            "paths": {}
        }
    
    # 构建完整的推理历史（前置条件Xp）
    # 收集跳步之前的所有步骤
    previous_steps = []
    for step in step_data:
        step_num = int(step['step_number'].split('_')[1]) if step['step_number'].startswith('step_') else 0
        if step_num < skipped_num:
            previous_steps.append(step['step_text'].strip())
    
    # 构建完整的前置条件Xp（确保上下文明确）
    if previous_steps:
        Xp = f"Problem: {problem}\n\nPrevious reasoning steps (already completed):\n" + "\n".join([f"{step}" for step in previous_steps])
    else:
        Xp = f"Problem: {problem}\n\nNo previous reasoning steps yet. You need to generate the first reasoning step."
    
    # 获取缺失的核心内容（确保引导精准）
    skipped_content = skipping_info['skipped_content'] if skipping_info['skipped_content'] != 'none' else "complete reasoning for this step"
    
    # 构建不同的引导式提示词Q'（强制格式+内容约束）
    guided_prompts = {
        "path_detailed_calculations": f"""{Xp}

Now you must generate Step {skipped_num} (the missing step).
Missing core content to include: {skipped_content}

REQUIREMENTS (MUST FOLLOW EXACTLY):
1. ONLY output Step {skipped_num} content. NO greetings, explanations, or extra text.
2. Format: "Step {skipped_num}: [detailed reasoning with calculations/intermediate steps]"
3. Must be relevant to the problem. NEVER say "missing context" or "can't help".
4. Keep it concise (1-3 sentences), focus on explicit calculations/quantitative reasoning.

Example Output (correct format):
Step {skipped_num}: Calculate the distance by speed × time (5km/h × 2h = 10km). Then confirm it matches the required route length.
""",
        
        "path_logical_deduction": f"""{Xp}

Now you must generate Step {skipped_num} (the missing step).
Missing core content to include: {skipped_content}

REQUIREMENTS (MUST FOLLOW EXACTLY):
1. ONLY output Step {skipped_num} content. NO extra explanations or greetings.
2. Format: "Step {skipped_num}: [logical deduction with if-then reasoning]"
3. Must connect to the problem/previous steps. NEVER mention missing context.
4. Focus on logical connections (e.g., cause-effect, necessary conditions), 1-3 sentences.

Example Output (correct format):
Step {skipped_num}: Since the building's east side is the entry point, and the question asks for other directions, we can eliminate east and focus on north/south/west.
""",
        
        "path_conceptual_analysis": f"""{Xp}

Now you must generate Step {skipped_num} (the missing step).
Missing core content to include: {skipped_content}

REQUIREMENTS (MUST FOLLOW EXACTLY):
1. ONLY output Step {skipped_num} content. NO extra text or greetings.
2. Format: "Step {skipped_num}: [conceptual analysis with underlying principles]"
3. Must relate to the problem's core concepts. NEVER say "incomplete context".
4. Concise (1-3 sentences), explain the principle guiding this reasoning step.

Example Output (correct format):
Step {skipped_num}: This step relies on the principle of direction elimination—removing known directions (east) leaves the remaining possible directions to analyze.
"""
    }
    
    results = {
        "status": "ok", 
        "actual_skip_hidden_shape": actual_skip_hidden.shape,
        "similarities": {}, 
        "paths": {},
        "generated_hidden_states": {},
        "variance_analysis": {}
    }
    
    # 为每条路径生成3个样本（N-best generations）
    for path_name, guided_prompt in guided_prompts.items():
        individual_sims = []
        valid_path_texts = []
        valid_generated_hiddens = []
        
        for sample_num in range(3):  # 3次采样，保证稳定性
            full_prompt = build_llama3_prompt(
                "You are a precise reasoning assistant. Generate only the required step following the exact format and requirements. Do NOT add any extra content.",
                guided_prompt
            )
            
            inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
            with torch.no_grad():
                path_outputs = model.generate(
                    **inputs,
                    max_new_tokens=100,  # 限制长度，避免冗余
                    do_sample=True,
                    temperature=0.6,  # 平衡随机性和一致性
                    top_p=0.9,  # 提升生成质量
                    output_hidden_states=True,
                    return_dict_in_generate=True,
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )
            
            # 解码并过滤无效内容
            generated_ids = path_outputs.sequences[0, len(inputs.input_ids[0]):]
            raw_text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
            
            # 严格过滤：只保留符合格式的Step N内容
            filtered_text = ""
            if f"Step {skipped_num}:" in raw_text:
                # 提取Step N开头的内容，直到换行或结束
                step_start = raw_text.find(f"Step {skipped_num}:")
                step_end = raw_text.find("\n", step_start) if "\n" in raw_text[step_start:] else len(raw_text)
                filtered_text = raw_text[step_start:step_end].strip()
                # 移除可能的多余标记（如模型添加的序号）
                filtered_text = re.sub(r'^Step\s*\d+:\s*', f'Step {skipped_num}: ', filtered_text)
            
            # 仅保留有效文本（非空且符合格式）
            if filtered_text and filtered_text.startswith(f"Step {skipped_num}:"):
                valid_path_texts.append(filtered_text)
                
                # 提取该样本的hidden state（取生成序列的最后一个token的隐藏状态）
                if hasattr(path_outputs, 'hidden_states') and path_outputs.hidden_states:
                    # 修正：LLaMA3的hidden_states结构是 tuple(生成步骤) → tuple(模型层) → tensor
                    # 1. 取最后一步生成的隐藏状态（tuple(模型层)）
                    last_step_hiddens = path_outputs.hidden_states[-1]  # tuple: (layer_0, layer_1, ..., layer_n)
                    # 2. 取最后一层的隐藏状态（tensor: (batch, seq_len, hidden_size)）
                    if isinstance(last_step_hiddens, tuple) and len(last_step_hiddens) > 0:
                        last_layer_hiddens = last_step_hiddens[-1]  # 最后一层的tensor
                        # 3. 验证tensor形状，取最后一个token的隐藏状态
                        if last_layer_hiddens.ndim == 3 and last_layer_hiddens.shape[1] > 0:
                            path_hidden = last_layer_hiddens[0, -1, :].cpu().numpy()  # (hidden_size,)
                            valid_generated_hiddens.append(path_hidden)
            else:
                print(f"⚠️  路径 {path_name} 样本 {sample_num+1} 生成无效内容，已过滤：{raw_text[:50]}...")
        
        # 处理无有效样本的情况
        if not valid_generated_hiddens:
            results["similarities"][path_name] = {
                "individual": [0.0, 0.0, 0.0],
                "mean": 0.0,
                "std": 0.0,
                "variance": 0.0
            }
            results["paths"][path_name] = ["无有效生成内容"]
            results["generated_hidden_states"][path_name] = [None, None, None]
            continue
        
        # 计算相似度（实际跳步 vs 有效生成路径）
        sims = calculate_similarity_between_paths(actual_skip_hidden, valid_generated_hiddens)
        # 补充空值以保证3个样本的结构一致性
        while len(sims) < 3:
            sims.append(0.0)
        
        # 计算统计信息
        mean_sim = float(np.mean(sims[:3]))  # 取前3个（最多3个）
        std_sim = float(np.std(sims[:3]))
        var_sim = float(np.var(sims[:3]))
        
        results["similarities"][path_name] = {
            "individual": sims[:3],  # 确保长度为3
            "mean": mean_sim,
            "std": std_sim,
            "variance": var_sim
        }
        results["paths"][path_name] = valid_path_texts + ["无有效生成内容"] * (3 - len(valid_path_texts))  # 补全3个
        results["generated_hidden_states"][path_name] = valid_generated_hiddens + [None] * (3 - len(valid_generated_hiddens))  # 补全3个
    
    # 计算跨路径的方差分析（所有路径的平均相似度的方差）
    all_path_means = []
    for path_name in guided_prompts.keys():
        path_mean = results["similarities"][path_name]["mean"]
        all_path_means.append(path_mean)
    
    cross_path_variance = float(np.var(all_path_means)) if len(all_path_means) > 1 else 0.0
    results["variance_analysis"]["cross_path_variance"] = cross_path_variance
    results["variance_analysis"]["path_means"] = all_path_means
    results["variance_analysis"]["path_names"] = list(guided_prompts.keys())
    
    return results

def print_detailed_similarity_analysis(skip_step, similarity_results, paths):
    """
    打印详细的相似度分析，包括每次相似度和方差分析
    """
    print(f"\n🔍 【跳步 {skip_step} 详细相似度分析】")
    print("-" * 60)
    
    # 打印各路径相似度
    for path_name, sim_data in similarity_results.items():
        individual_sims = sim_data["individual"]
        mean_sim = sim_data["mean"]
        variance = sim_data["variance"]
        
        print(f"\n📊 {path_name.replace('_', ' ').title()}:")
        print(f"   单次相似度: {[f'{sim:.4f}' for sim in individual_sims]}")
        print(f"   平均相似度: {mean_sim:.4f}")
        print(f"   样本内方差: {variance:.6f}")
        
        # 显示对应的路径文本（只显示有效内容）
        path_texts = paths[path_name]
        for i, text in enumerate(path_texts):
            if text != "无有效生成内容":
                print(f"   样本{i+1}: {text}")
            else:
                print(f"   样本{i+1}: {text}")
    
    # 跨路径方差分析
    cross_variance = similarity_results.get("variance_analysis", {}).get("cross_path_variance", 0.0)
    path_means = similarity_results.get("variance_analysis", {}).get("path_means", [])
    path_names = similarity_results.get("variance_analysis", {}).get("path_names", [])
    
    print(f"\n🎯 跨路径方差分析:")
    print(f"   各路径平均相似度: {dict(zip(path_names, [f'{m:.4f}' for m in path_means]))}")
    print(f"   跨路径方差: {cross_variance:.6f}")
    
    # 方差解读（结合实际场景优化）
    if cross_variance < 0.005:
        print("   📝 解读: 极低方差 - 跳步状态与所有路径相似度接近，可能是推理空间狭窄或跳步状态信息泛化")
    elif 0.005 <= cross_variance < 0.02:
        print("   📝 解读: 低方差 - 跳步状态对多条路径有弱偏好，无明显倾向性")
    elif 0.02 <= cross_variance < 0.05:
        print("   📝 解读: 中等方差 - 跳步状态对某条路径有明显偏好，支持潜在推理路径假设")
    else:
        print("   📝 解读: 高方差 - 跳步状态与特定路径高度吻合，强烈支持潜在推理路径假设")

In [5]:
# =====================================================
# 单个问题实验流程
# =====================================================
def main_experiment_with_save(test_sample):
    """按照方法一要求修改的实验流程"""
    test_id = test_sample['id']
    problem = test_sample['problem']
    expected_steps = test_sample['expected_complete_reasoning']
    
    # 生成分步推理（关键：在generate中设置output_hidden_states=True）
    step_by_step_prompt = build_llama3_prompt(
        "You solve reasoning problems in a FIXED FORMAT. Follow EXACTLY: 1. Start with 'Step 1: ...' 2. Each step has only core logic 3. End with 'Final Answer: ...' 4. No extra greetings/explanations 5. Do NOT ask for more information (try your best if problem is simple)",
        f"Problem: {problem}\n\nSolve step by step in the FIXED FORMAT (only core logic, no extra words):"
    )
    inputs = tokenizer(step_by_step_prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(model.device)
    with torch.no_grad():
        # 关键修正：在generate中设置output_hidden_states=True
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            num_return_sequences=1,
            do_sample=True,
            output_hidden_states=True,  # 正确的位置
            return_dict_in_generate=True,
            pad_token_id=tokenizer.eos_token_id
        )
    full_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    step_by_step_response = full_response.split("assistant")[-1].strip() if "assistant" in full_response else full_response
    
    # 提取步骤级hidden states
    prompt_length = inputs.input_ids.shape[1]
    step_data = extract_step_level_hidden_states(outputs, prompt_length, step_by_step_response)
    
    # 识别跳步位置
    skipping_info, judge_analysis = identify_skipped_steps(step_by_step_response, problem, expected_steps)
    
    # 为每个跳步生成显式参考路径并计算相似度
    all_skip_results = []
    if skipping_info['skipped_steps']:
        for skip_step in skipping_info['skipped_steps']:
            # 关键修改：直接传递完整的skipping_info，不再构建single_skip_info
            comparison_results = generate_explicit_reference_paths(
                skipping_info,  # 直接传递完整的跳步信息
                step_data, 
                problem,
                tokenizer, 
                model,
                outputs,
                prompt_length
            )
            all_skip_results.append({
                'skip_step': skip_step,
                'similarity_results': comparison_results.get('similarities', {}),
                'variance_analysis': comparison_results.get('variance_analysis', {}),
                'paths': comparison_results.get('paths', {}),
                'actual_hidden_shape': comparison_results.get('actual_skip_hidden_shape', 'unknown'),
                'status': comparison_results.get('status', 'unknown')
            })
    
    # 构建完整结果
    result = {
        'test_id': test_id,
        'problem': problem,
        'step_by_step_response': step_by_step_response,
        'judge_analysis': judge_analysis,
        'skipping_info': skipping_info,
        'all_skip_results': all_skip_results,
        'step_data': step_data,
        'prompt_length': prompt_length
    }
    
    # 保存数据
    save_experiment1_data(result)
    
    return result

# =====================================================
# 修正的详细相似度分析函数
# =====================================================
def print_detailed_similarity_analysis(skip_step, skip_result):
    """
    修正版：打印详细的相似度分析，包括每次相似度和方差分析
    关键修复：正确获取跨路径方差数据
    """
    similarity_results = skip_result.get('similarity_results', {})
    paths = skip_result.get('paths', {})
    variance_analysis = skip_result.get('variance_analysis', {})
    
    print(f"\n🔍 【跳步 {skip_step} 详细相似度分析】")
    print("-" * 60)
    
    # 打印各路径相似度
    for path_name, sim_data in similarity_results.items():
        individual_sims = sim_data["individual"]
        mean_sim = sim_data["mean"]
        variance = sim_data["variance"]
        
        print(f"\n📊 {path_name.replace('_', ' ').title()}:")
        print(f"   单次相似度: {[f'{sim:.4f}' for sim in individual_sims]}")
        print(f"   平均相似度: {mean_sim:.4f}")
        print(f"   样本内方差: {variance:.6f}")
        
        # 显示对应的路径文本（只显示有效内容）
        path_texts = paths.get(path_name, [])
        for i, text in enumerate(path_texts):
            if text != "无有效生成内容":
                print(f"   样本{i+1}: {text}")
            else:
                print(f"   样本{i+1}: {text}")
    
    # 跨路径方差分析 - 关键修复：直接从variance_analysis获取
    cross_variance = variance_analysis.get("cross_path_variance", 0.0)
    path_means = variance_analysis.get("path_means", [])
    path_names = variance_analysis.get("path_names", [])
    
    print(f"\n🎯 跨路径方差分析:")
    path_mean_dict = dict(zip(path_names, [f'{m:.4f}' for m in path_means]))
    print(f"   各路径平均相似度: {path_mean_dict}")
    print(f"   跨路径方差: {cross_variance:.6f}")
    
    # 方差解读（结合实际场景优化）
    if cross_variance < 0.005:
        print("   📝 解读: 极低方差 - 跳步状态与所有路径相似度接近，可能是推理空间狭窄或跳步状态信息泛化")
    elif 0.005 <= cross_variance < 0.02:
        print("   📝 解读: 低方差 - 跳步状态对多条路径有弱偏好，无明显倾向性")
    elif 0.02 <= cross_variance < 0.05:
        print("   📝 解读: 中等方差 - 跳步状态对某条路径有明显偏好，支持潜在推理路径假设")
    else:
        print("   📝 解读: 高方差 - 跳步状态与特定路径高度吻合，强烈支持潜在推理路径假设")

# =====================================================
# 批量运行测试集（更新显示逻辑）
# =====================================================
def run_experiment1_with_save():
    """运行实验一并保存所有数据"""
    testset = load_testset("/kaggle/input/testset/testset.csv")
    
    print("=" * 70)
    print(f"实验一：基于隐藏状态相似性的路径匹配分析 - 共 {len(testset)} 个问题")
    print("=" * 70)
    
    # 路径描述映射
    path_descriptions = {
        "path_detailed_calculations": "详细计算路径",
        "path_logical_deduction": "逻辑推理路径", 
        "path_conceptual_analysis": "概念分析路径"
    }
    
    all_results = []
    for i, test_sample in enumerate(testset):
        print(f"\n处理进度: {i+1}/{len(testset)}")
        result = main_experiment_with_save(test_sample)
        all_results.append(result)
        
        test_id = result['test_id']
        skipping_info = result['skipping_info']
        all_skip_results = result['all_skip_results']
        step_by_step_response = result['step_by_step_response']
        judge_analysis = result['judge_analysis']
        
        print(f"\n【{test_id} 验证结果】")
        print(f"模型推理过程：\n{step_by_step_response[:300]}...")
        print(f"\nLLM跳步分析：\n{judge_analysis}")
        
        # 输出所有跳步的详细相似度分析
        if all_skip_results:
            for skip_result in all_skip_results:
                skip_step = skip_result['skip_step']
                similarities = skip_result['similarity_results']
                
                # 关键修复：传递完整的skip_result而不是分开的参数
                print_detailed_similarity_analysis(skip_step, skip_result)
                
                # 找出最佳匹配路径
                if similarities:
                    best_path = max(similarities.items(), key=lambda x: x[1]['mean'])[0]
                    best_similarity = similarities[best_path]['mean']
                    best_path_desc = path_descriptions.get(best_path, best_path)
                    print(f"\n🎯 {test_id} {skip_step} 最佳匹配: {best_path_desc} (相似度: {best_similarity:.4f})")
        else:
            print(f"\n{test_id}: 无跳步（根据Judge分析）")
    
    # 汇总数据
    summarize_experiment1_data()
    
    # 保存完整数据集
    if save_complete_experiment1_data():
        print(f"\n📊 实验一数据统计:")
        print(f"  总测试案例: {len(experiment1_data)}")
        total_steps = sum(len(data['step_data']) for data in experiment1_data.values())
        total_skips = sum(len(data['skipping_info']['skipped_steps']) for data in experiment1_data.values())
        print(f"  总推理步骤: {total_steps}")
        print(f"  总跳步数量: {total_skips}")
        
        # 添加相似度统计
        valid_comparisons = 0
        total_similarity = 0.0
        variance_data = []
        
        for test_id, data in experiment1_data.items():
            if data.get('all_skip_results'):
                for skip_result in data['all_skip_results']:
                    similarities = skip_result.get('similarity_results', {})
                    variance_analysis = skip_result.get('variance_analysis', {})
                    
                    # 收集跨路径方差数据
                    cross_variance = variance_analysis.get("cross_path_variance", 0.0)
                    variance_data.append(cross_variance)
                    
                    for path_data in similarities.values():
                        if path_data['mean'] > 0:
                            valid_comparisons += 1
                            total_similarity += path_data['mean']
        
        if valid_comparisons > 0:
            avg_similarity = total_similarity / valid_comparisons
            print(f"  有效对比次数: {valid_comparisons}")
            print(f"  平均相似度: {avg_similarity:.4f}")
        
        if variance_data:
            avg_variance = np.mean(variance_data)
            print(f"  平均跨路径方差: {avg_variance:.6f}")
            print(f"  方差数据范围: [{min(variance_data):.6f}, {max(variance_data):.6f}]")
    
    print(f"\n🎉 实验一完成！所有数据已保存")
    return all_results

# =====================================================
# 执行实验一
# =====================================================
if __name__ == "__main__":
    # 运行实验一并保存数据
    experiment1_results = run_experiment1_with_save()
    
    # 显示保存的文件
    print("\n📁 保存的文件列表:")
    for file in os.listdir('.'):
        if file.startswith('experiment1_') and file.endswith('.pkl'):
            file_size = os.path.getsize(file) / 1024
            print(f"  {file} ({file_size:.1f} KB)")

✓ 成功加载 8 个测试问题
实验一：基于隐藏状态相似性的路径匹配分析 - 共 8 个问题

处理进度: 1/8


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✓ 已保存 Test1 的hidden states数据到 experiment1_hidden_states_Test1.pkl
  - 步骤数量: 5
  - 跳步数量: 1
  - hidden state形状: (4096,)
  - 提取层索引: -1

【Test1 验证结果】
模型推理过程：
Step 1: You are facing north.

Step 2: If you walk 10 meters, you will be 10 meters north.

Step 3: If you then turn 90 degrees to your right, you will be facing east.

Step 4: If you walk 10 meters, you will be 10 meters east.

Final Answer: You are 10 meters east and 10 meters north of the entranc...

LLM跳步分析：
SKIPPED_STEPS: 1
SKIPPED_CONTENT: walking west
SKIPPING_LOCATION: start

🔍 【跳步 step_1 详细相似度分析】
------------------------------------------------------------

📊 Path Detailed Calculations:
   单次相似度: ['0.8102', '0.8408', '0.8048']
   平均相似度: 0.8186
   样本内方差: 0.000250
   样本1: Step 1: Walk 10 meters west to reach the entrance.
   样本2: Step 1: Walk west for 10 meters to reach the entrance, since you are currently 10 meters east of the entrance.
   样本3: Step 1: Since you are 10 meters east of the entrance, walk west by 10 meters to re

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✓ 已保存 Test2 的hidden states数据到 experiment1_hidden_states_Test2.pkl
  - 步骤数量: 4
  - 跳步数量: 0
  - hidden state形状: (4096,)
  - 提取层索引: -1

【Test2 验证结果】
模型推理过程：
Step 1: All bachelors have never married.
Step 2: John is a man who has never been married.
Step 3: Since John has never been married, he is a bachelor.
Final Answer: Yes....

LLM跳步分析：
SKIPPED_STEPS: none
SKIPPED_CONTENT: none
SKIPPING_LOCATION: none

Test2: 无跳步（根据Judge分析）

处理进度: 3/8


...

✓ 已保存 Test6 的hidden states数据到 experiment1_hidden_states_Test6.pkl
  - 步骤数量: 4
  - 跳步数量: 1
  - hidden state形状: (4096,)
  - 提取层索引: -1

【Test6 验证结果】
模型推理过程：
Step 1: Alabama's capital is Montgomery.

Step 2: Alabama's state bird is Yellowhammer.

Step 3: The Yellowhammer is a type of Warbler.

Final Answer: The state capital is Montgomery....

LLM跳步分析：
SKIPPED_STEPS: 1
SKIPPED_CONTENT: Juneau
SKIPPING_LOCATION: start

🔍 【跳步 step_1 详细相似度分析】
------------------------------------------------------------

📊 Path Detailed Calculations:
   单次相似度: ['0.7549', '0.8276', '0.8089']
   平均相似度: 0.7972
   样本内方差: 0.000949
   样本1: Step 1: Determine the state by analyzing the given information, noting that Juneau is the capital of Alaska, and Montgomery is the capital of Alabama.
   样本2: Step 1: Identify the state associated with the capital Juneau, which is Alaska, and note that Montgomery is not the capital of Alaska.
   样本3: Step 1: Identify the state with capital Juneau, which is Alaska, and note that it

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✓ 已保存 Test7 的hidden states数据到 experiment1_hidden_states_Test7.pkl
  - 步骤数量: 10
  - 跳步数量: 1
  - hidden state形状: (4096,)
  - 提取层索引: -1

【Test7 验证结果】
模型推理过程：
Step 1: Write the given expression as (33ⁿ - 16ⁿ) - (28ⁿ - 11ⁿ).

Step 2: Factor the first term as (33ⁿ - 16ⁿ) = (33ⁿ - 16ⁿ + 0).

Step 3: Factor the second term as (28ⁿ - 11ⁿ) = (28ⁿ - 16ⁿ + 8ⁿ).

Step 4: Combine the two factored terms as (33ⁿ - 16ⁿ) - (28ⁿ - 11ⁿ) = (33ⁿ - 16ⁿ + 0) - (28ⁿ - 16ⁿ + 8...

LLM跳步分析：
SKIPPED_STEPS: 1
SKIPPED_CONTENT: Expected Step 1: Factor 85 into prime factors: 85 = 5 × 17 (coprime
SKIPPING_LOCATION: start

🔍 【跳步 step_1 详细相似度分析】
------------------------------------------------------------

📊 Path Detailed Calculations:
   单次相似度: ['0.8565', '0.9048', '0.8162']
   平均相似度: 0.8592
   样本内方差: 0.001313
   样本1: Step 1: 85 = 5 × 17, so 85 is the product of two coprime numbers, which means they share no common factors other than 1.
   样本2: Step 1: Factor 85 into prime factors: 85 = 5 × 17, which are coprime.
   样本

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✓ 已保存 Test8 的hidden states数据到 experiment1_hidden_states_Test8.pkl
  - 步骤数量: 6
  - 跳步数量: 1
  - hidden state形状: (4096,)
  - 提取层索引: -1

【Test8 验证结果】
模型推理过程：
Step 1: I am driving.

Step 2: I am moving my vehicle.

Step 3: I am using a mode of transportation.

Step 4: I am likely on a road.

Step 5: I am probably using a vehicle.

Final Answer: I am driving a vehicle on a road....

LLM跳步分析：
SKIPPED_STEPS: 1
SKIPPED_CONTENT: hit on the brakes
SKIPPING_LOCATION: start

🔍 【跳步 step_1 详细相似度分析】
------------------------------------------------------------

📊 Path Detailed Calculations:
   单次相似度: ['0.7966', '0.6269', '0.8211']
   平均相似度: 0.7482
   样本内方差: 0.007458
   样本1: Step 1: As I am driving a vehicle on a road, I hit on the brakes to slow down or stop.
   样本2: Step 1: As I am driving a vehicle on a road, I need to adjust my speed to match the road conditions.
   样本3: Step 1: As I am driving a vehicle on a road, I hit the brakes to slow down or stop, which indicates I am controlling the vehicle's